In [ ]:
# Libs
import os
import pandas as pd
from download_transformacao_CNPJ import EXTRATOR_CNPJ
from pyspark.sql.functions import concat_ws, lpad, coalesce, when, lit
from time import localtime, strftime
current_dir = os.getcwd()
dir_dados = os.path.join(current_dir, r"dados")

In [ ]:
# Se passar baixar_e_extrair como false, precisa do nome do arquivo.
print(f'Começando a buscar os dados: {strftime("%d/%m/%Y %H:%M:%S", localtime())}')
ESTABELECIMENTOS, spark = EXTRATOR_CNPJ(baixar_e_extrair=True, nome_arquivo="Estabelecimentos").run()
print(f'Termino da coleta dos ESTABELECIMENTOS: {strftime("%d/%m/%Y %H:%M:%S", localtime())}')
EMPRESAS, spark = EXTRATOR_CNPJ(baixar_e_extrair=True, nome_arquivo="Empresas").run()
print(f'Termino da coleta dos EMPRESAS: {strftime("%d/%m/%Y %H:%M:%S", localtime())}')
MUNICIPIOS, spark = EXTRATOR_CNPJ(baixar_e_extrair=True, nome_arquivo="Municipios").run()
print(f'Final da coleta dos MUNICIPIOS: {strftime("%d/%m/%Y %H:%M:%S", localtime())}')
CNAES, spark = EXTRATOR_CNPJ(baixar_e_extrair=True, nome_arquivo="Cnaes").run()
print(f'Final da coleta dos dados: {strftime("%d/%m/%Y %H:%M:%S", localtime())}')

In [ ]:
from pyspark.sql.functions import split, explode
# Cria uma nova coluna com os valores da coluna CNAE_SECUNDARIO divididos em uma lista
ESTABELECIMENTOS = ESTABELECIMENTOS.withColumn("CNAE_SECUNDARIO_LIST", split(ESTABELECIMENTOS["CNAE_SECUNDARIO"], ","))
# Explode a coluna CNAE_SECUNDARIO_LIST para criar uma linha para cada valor
ESTABELECIMENTOS = ESTABELECIMENTOS.select("*", explode("CNAE_SECUNDARIO_LIST").alias("CNAE_SECUNDARIO_VALUE"))

# Cria uma view com o mesmo nome do DataFrame
ESTABELECIMENTOS.createOrReplaceTempView("ESTABELECIMENTOS")
EMPRESAS.createOrReplaceTempView("EMPRESAS")
MUNICIPIOS.createOrReplaceTempView("MUNICIPIOS")
CNAES.createOrReplaceTempView("CNAES")

In [ ]:
#ESTABELECIMENTOS.show(1, vertical=True)
print(ESTABELECIMENTOS.count())

In [ ]:
CNAES_FILTROS = {
        5620104:'Fornecimento de alimentos preparados preponderantemente para consumo domiciliar',
        5611201:'Restaurantes e similares',
        5611203:'Lanchonetes casas de chá de sucos e similares',
        5611204:'Bares e outros estabelecimentos especializados em servir bebidas sem entretenimento',
        5611205:'Bares e outros estabelecimentos especializados em servir bebidas com entretenimento',
        4721102:'Padaria e confeitaria com predominância de revenda',
        5612100:'Serviços ambulantes de alimentação'
        }

In [ ]:
def filtra_cnpj_cnae_principal():
    dados = spark.sql(
        """SELECT
          CONCAT(LPAD(ESTABELECIMENTOS.CNPJ_BASE, 8, '0'), LPAD(ESTABELECIMENTOS.CNPJ_ORDEM, 4, '0'), LPAD(ESTABELECIMENTOS.CNPJ_DV, 2, '0')) AS CNPJ,
          EMPRESAS.RAZAO_SOCIAL,
          ESTABELECIMENTOS.NOME_FANTASIA,
          ESTABELECIMENTOS.SITUACAO_CADASTRAL,
          ESTABELECIMENTOS.DATA_SITUACAO_CADASTRAL,
          ESTABELECIMENTOS.DATA_INICIO_ATIVIDADE,
          ESTABELECIMENTOS.CNAE_PRINCIPAL,
          CNAES.DESCRICAO_CNAE,
          ESTABELECIMENTOS.CNAE_SECUNDARIO,
          CONCAT(
            COALESCE(ESTABELECIMENTOS.TIPO_LOGRADOURO, ''),
            ' ',
            COALESCE(ESTABELECIMENTOS.LOGRADOURO, ''),
            ' ',
            COALESCE(ESTABELECIMENTOS.NUMERO, ''),
            ' ',
            COALESCE(ESTABELECIMENTOS.COMPLEMENTO, '')
          ) AS ENDERECO,
          ESTABELECIMENTOS.BAIRRO,
          MUNICIPIOS.NOME_MUNICIPIO AS CIDADE,
          ESTABELECIMENTOS.UF,
          ESTABELECIMENTOS.CEP,
          CONCAT(
            COALESCE(ESTABELECIMENTOS.DDD_CONTATO, ''),
            ' ',
            COALESCE(ESTABELECIMENTOS.TELEFONE_CONTATO, '')
          ) AS TELEFONE,
          ESTABELECIMENTOS.EMAIL
        FROM ESTABELECIMENTOS
        LEFT JOIN EMPRESAS ON ESTABELECIMENTOS.CNPJ_BASE = EMPRESAS.CNPJ_BASE
        LEFT JOIN MUNICIPIOS ON ESTABELECIMENTOS.CODIGO_MUNICIPIO = MUNICIPIOS.CODIGO_MUNICIPIO
        LEFT JOIN CNAES ON ESTABELECIMENTOS.CNAE_PRINCIPAL = CNAES.CODIGO_CNAE
        WHERE ESTABELECIMENTOS.CNAE_PRINCIPAL IN ('4721102', '5611201', '5611203', '5611204', '5611205',  '5612100', '5620104') AND ESTABELECIMENTOS.SITUACAO_CADASTRAL IN (2, 3, 4)
        ORDER BY ESTABELECIMENTOS.CNPJ_BASE
    """)
    return dados

def filtra_cnpj_cnae_segundario():
    dados = spark.sql(
        """SELECT
          CONCAT(LPAD(ESTABELECIMENTOS.CNPJ_BASE, 8, '0'), LPAD(ESTABELECIMENTOS.CNPJ_ORDEM, 4, '0'), LPAD(ESTABELECIMENTOS.CNPJ_DV, 2, '0')) AS CNPJ,
          EMPRESAS.RAZAO_SOCIAL,
          ESTABELECIMENTOS.NOME_FANTASIA,
          ESTABELECIMENTOS.SITUACAO_CADASTRAL,
          ESTABELECIMENTOS.DATA_SITUACAO_CADASTRAL,
          ESTABELECIMENTOS.DATA_INICIO_ATIVIDADE,
          ESTABELECIMENTOS.CNAE_PRINCIPAL,
          CNAES.DESCRICAO_CNAE,
          ESTABELECIMENTOS.CNAE_SECUNDARIO,
          CONCAT(
            COALESCE(ESTABELECIMENTOS.TIPO_LOGRADOURO, ''),
            ' ',
            COALESCE(ESTABELECIMENTOS.LOGRADOURO, ''),
            ' ',
            COALESCE(ESTABELECIMENTOS.NUMERO, ''),
            ' ',
            COALESCE(ESTABELECIMENTOS.COMPLEMENTO, '')
          ) AS ENDERECO,
          ESTABELECIMENTOS.BAIRRO,
          MUNICIPIOS.NOME_MUNICIPIO AS CIDADE,
          ESTABELECIMENTOS.UF,
          ESTABELECIMENTOS.CEP,
          CONCAT(
            COALESCE(ESTABELECIMENTOS.DDD_CONTATO, ''),
            ' ',
            COALESCE(ESTABELECIMENTOS.TELEFONE_CONTATO, '')
          ) AS TELEFONE,
          ESTABELECIMENTOS.EMAIL
        FROM ESTABELECIMENTOS
        LEFT JOIN EMPRESAS ON ESTABELECIMENTOS.CNPJ_BASE = EMPRESAS.CNPJ_BASE
        LEFT JOIN MUNICIPIOS ON ESTABELECIMENTOS.CODIGO_MUNICIPIO = MUNICIPIOS.CODIGO_MUNICIPIO
        LEFT JOIN CNAES ON ESTABELECIMENTOS.CNAE_PRINCIPAL = CNAES.CODIGO_CNAE
        WHERE ESTABELECIMENTOS.CNAE_SECUNDARIO_VALUE IN ('4721102', '5611201', '5611203', '5611204', '5611205',  '5612100', '5620104') AND ESTABELECIMENTOS.SITUACAO_CADASTRAL IN (2, 3, 4)
        ORDER BY ESTABELECIMENTOS.CNPJ_BASE
    """)
    return dados

In [ ]:
dados_1 = filtra_cnpj_cnae_principal()
print(dados_1.count())
dados_2 = filtra_cnpj_cnae_segundario()
print(dados_2.count())

In [ ]:
# Importa a função de backup
from backup_limpeza import backup_limpeza_simples
# Mapeia o caminho para salvar o arquivo, verifica se já existe dados lá e faz o backup
arquivo_csv = os.path.join(dir_dados, r"csv\BASE_RFB.csv")
if os.path.exists(arquivo_csv):
    nome_backup = dir_dados + r"/backup/"
    if not os.path.exists(nome_backup):
        os.makedirs(nome_backup)
    backup_limpeza_simples(pasta=arquivo_csv.replace(r"BASE_RFB.csv", ""), nome_zipado=nome_backup + f"BASE_RFB_{strftime('%d-%m-%Y %H_%M_%S', localtime())}.zip")

# Chama a função de filtrar por cnae
dados = filtra_cnpj_cnae_principal()
# Converte para pandas dataframe
dados_pandas = dados.toPandas()
dados_pandas.rename(columns={"DESCRICAO_CNAE":'CNAE_DESCRICAO'}, inplace=True)
# Seleciona os campos que vou usar no final
dados_pandas = dados_pandas[["CNPJ","RAZAO_SOCIAL","NOME_FANTASIA",
    "SITUACAO_CADASTRAL","DATA_SITUACAO_CADASTRAL",
    "DATA_INICIO_ATIVIDADE","CNAE_PRINCIPAL","ENDERECO",
    "BAIRRO","CIDADE","UF","CEP","TELEFONE","CNAE_DESCRICAO", "EMAIL"]]
# Só pra contar quantos dados vieram
print(len(dados_pandas.index))
# Salvo o arquivo em um cnv
dados_pandas.to_csv(arquivo_csv, sep=";",mode="a", encoding="utf-8", index=False)

# Chama a função de filtrar por cnae
dados = filtra_cnpj_cnae_segundario()
# Converte para pandas dataframe
dados_pandas = dados.toPandas()
dados_pandas.rename(columns={"DESCRICAO_CNAE":'CNAE_DESCRICAO'}, inplace=True)
# Seleciona os campos que vou usar no final
dados_pandas = dados_pandas[["CNPJ","RAZAO_SOCIAL","NOME_FANTASIA",
    "SITUACAO_CADASTRAL","DATA_SITUACAO_CADASTRAL",
    "DATA_INICIO_ATIVIDADE","CNAE_PRINCIPAL","ENDERECO",
    "BAIRRO","CIDADE","UF","CEP","TELEFONE","CNAE_DESCRICAO", "EMAIL"]]
# Só pra contar quantos dados vieram
print(len(dados_pandas.index))
# Salvo o arquivo em um cnv
dados_pandas.to_csv(arquivo_csv, sep=";",mode="a", encoding="utf-8", index=False)

In [ ]:
def filtragem_cnae_sql(cod_cnae: int):
    # Cria uma view com o mesmo nome do DataFrame
    ESTABELECIMENTOS.createOrReplaceTempView("ESTABELECIMENTOS")
    EMPRESAS.createOrReplaceTempView("EMPRESAS")
    MUNICIPIOS.createOrReplaceTempView("MUNICIPIOS")
    # cria um dataframe com base na query
    dataframe = spark.sql(
        f"""
        SELECT CONCAT(LPAD(EST.CNPJ_BASE, 8, '0'), LPAD(EST.CNPJ_ORDEM, 4, '0'), LPAD(EST.CNPJ_DV, 2, '0')) AS CNPJ,
                EMP.RAZAO_SOCIAL,
                EST.NOME_FANTASIA,
                EST.SITUACAO_CADASTRAL,
                EST.DATA_SITUACAO_CADASTRAL,
                EST.DATA_INICIO_ATIVIDADE,
                EST.CNAE_PRINCIPAL,
                CONCAT(
                    COALESCE(EST.TIPO_LOGRADOURO, ''),
                    ' ',
                    COALESCE(EST.LOGRADOURO, ''),
                    ' ',
                    COALESCE(EST.NUMERO, ''),
                    ' ',
                    COALESCE(EST.COMPLEMENTO, '')
                ) AS ENDERECO
                EST.BAIRRO,
                MUN.NOME_MUNICIPIO AS CIDADE,
                EST.UF,
                EST.CEP,
                CONCAT(
                    COALESCE(EST.DDD_CONTATO, ''), 
                    ' ',
                    COALESCE(EST.TELEFONE_CONTATO, '')
                    ) AS TELEFONE,
                EST.EMAIL
        FROM ESTABELECIMENTOS AS EST, EMPRESAS AS EMP, MUNICIPIOS AS MUN
        WHERE EST.CNPJ_BASE = EMP.CNPJ_BASE
            AND EST.CODIGO_MUNICIPIO = MUN.CODIGO_MUNICIPIO
            AND EST.CNAE_PRINCIPAL = {cod_cnae}
            AND EST.SITUACAO_CADASTRAL IN (2, 3, 4)
        """
    )
    #display(dataframe.show(5))
    return dataframe

In [ ]:
def filtragem_cnae_df(cod_cnae:int):
    # cria um dataframe com base nos filtros aplicados
    from pyspark.sql.functions import concat_ws, lpad, coalesce, when, array_contains
    dataframe = (
        ESTABELECIMENTOS
        .join(EMPRESAS, "CNPJ_BASE", "left")
        .join(MUNICIPIOS, "CODIGO_MUNICIPIO", "left")
        .where(
            (ESTABELECIMENTOS["CNAE_PRINCIPAL"] == f"{cod_cnae}") &
            (ESTABELECIMENTOS["SITUACAO_CADASTRAL"].isin([2, 3, 4]))
        )
        .select(
            concat_ws("", 
                lpad(ESTABELECIMENTOS["CNPJ_BASE"].cast("bigint"), 8, "0"), 
                lpad(ESTABELECIMENTOS["CNPJ_ORDEM"], 4, "0"), 
                lpad(ESTABELECIMENTOS["CNPJ_DV"], 2, "0")
            ).alias("CNPJ"),
            when(EMPRESAS.RAZAO_SOCIAL.isNull(), None).otherwise(EMPRESAS.RAZAO_SOCIAL).alias("RAZAO_SOCIAL"),
            when(ESTABELECIMENTOS.NOME_FANTASIA.isNull(), None).otherwise(ESTABELECIMENTOS.NOME_FANTASIA).alias("NOME_FANTASIA"),
            ESTABELECIMENTOS.SITUACAO_CADASTRAL,
            ESTABELECIMENTOS.DATA_SITUACAO_CADASTRAL,
            ESTABELECIMENTOS.DATA_INICIO_ATIVIDADE,
            ESTABELECIMENTOS.CNAE_PRINCIPAL,
            ESTABELECIMENTOS.CNAE_SECUNDARIO,
            concat_ws(" ",
                when(ESTABELECIMENTOS.TIPO_LOGRADOURO.isNull(), None).otherwise(ESTABELECIMENTOS.TIPO_LOGRADOURO),
                when(ESTABELECIMENTOS.LOGRADOURO.isNull(), None).otherwise(ESTABELECIMENTOS.LOGRADOURO),
                when(ESTABELECIMENTOS.NUMERO.isNull(), None).otherwise(ESTABELECIMENTOS.NUMERO),
                when(ESTABELECIMENTOS.COMPLEMENTO.isNull(), None).otherwise(ESTABELECIMENTOS.COMPLEMENTO)
            ).alias("ENDERECO"),
            ESTABELECIMENTOS.BAIRRO,
            MUNICIPIOS.NOME_MUNICIPIO.alias("CIDADE"),
            ESTABELECIMENTOS.UF,
            ESTABELECIMENTOS.CEP,
            concat_ws("-", 
                when(ESTABELECIMENTOS.DDD_CONTATO.isNull(), None).otherwise(ESTABELECIMENTOS.DDD_CONTATO),
                when(ESTABELECIMENTOS.TELEFONE_CONTATO.isNull(), None).otherwise(ESTABELECIMENTOS.TELEFONE_CONTATO)
            ).alias("TELEFONE"),
            ESTABELECIMENTOS.EMAIL
        )
    )

    #display(dataframe.show(5))
    return dataframe

In [ ]:
def filtra_cnae_lista(lista_cnaes:list):
    # cria um dataframe com base nos filtros aplicados
    from pyspark.sql.functions import concat_ws, lpad, coalesce, when
    dataframe = (
        ESTABELECIMENTOS
        .join(EMPRESAS, "CNPJ_BASE", "left")
        .join(MUNICIPIOS, "CODIGO_MUNICIPIO", "left")
        .where(
            (ESTABELECIMENTOS["CNAE_PRINCIPAL"].isin(lista_cnaes)) &
            (ESTABELECIMENTOS["SITUACAO_CADASTRAL"].isin([2, 3, 4]))
        )
        .select(
            concat_ws("", 
                lpad(ESTABELECIMENTOS["CNPJ_BASE"].cast("bigint"), 8, "0"), 
                lpad(ESTABELECIMENTOS["CNPJ_ORDEM"], 4, "0"), 
                lpad(ESTABELECIMENTOS["CNPJ_DV"], 2, "0")
            ).alias("CNPJ"),
            when(EMPRESAS.RAZAO_SOCIAL.isNull(), None).otherwise(EMPRESAS.RAZAO_SOCIAL).alias("RAZAO_SOCIAL"),
            when(ESTABELECIMENTOS.NOME_FANTASIA.isNull(), None).otherwise(ESTABELECIMENTOS.NOME_FANTASIA).alias("NOME_FANTASIA"),
            ESTABELECIMENTOS.SITUACAO_CADASTRAL,
            ESTABELECIMENTOS.DATA_SITUACAO_CADASTRAL,
            ESTABELECIMENTOS.DATA_INICIO_ATIVIDADE,
            ESTABELECIMENTOS.CNAE_PRINCIPAL,
            ESTABELECIMENTOS.CNAE_SECUNDARIO,
            concat_ws(" ",
                when(ESTABELECIMENTOS.TIPO_LOGRADOURO.isNull(), None).otherwise(ESTABELECIMENTOS.TIPO_LOGRADOURO),
                when(ESTABELECIMENTOS.LOGRADOURO.isNull(), None).otherwise(ESTABELECIMENTOS.LOGRADOURO),
                when(ESTABELECIMENTOS.NUMERO.isNull(), None).otherwise(ESTABELECIMENTOS.NUMERO),
                when(ESTABELECIMENTOS.COMPLEMENTO.isNull(), None).otherwise(ESTABELECIMENTOS.COMPLEMENTO)
            ).alias("ENDERECO"),
            ESTABELECIMENTOS.BAIRRO,
            MUNICIPIOS.NOME_MUNICIPIO.alias("CIDADE"),
            ESTABELECIMENTOS.UF,
            ESTABELECIMENTOS.CEP,
            concat_ws("-", 
                when(ESTABELECIMENTOS.DDD_CONTATO.isNull(), None).otherwise(ESTABELECIMENTOS.DDD_CONTATO),
                when(ESTABELECIMENTOS.TELEFONE_CONTATO.isNull(), None).otherwise(ESTABELECIMENTOS.TELEFONE_CONTATO)
            ).alias("TELEFONE"),
            ESTABELECIMENTOS.EMAIL
        )
    )

    #display(dataframe.show(5))
    return dataframe

In [ ]:
def filtra_cnae_lista_sql(lista_cnaes):
  tupla_strings = tuple(str(cnae) for cnae in lista_cnaes)
  
  """
  Filtra os estabelecimentos com base em uma lista de CNAEs.

  Args:
    lista_cnaes: Lista de CNAEs.

  Returns:
    DataFrame com os estabelecimentos filtrados.
  """
  from pyspark.sql.functions import concat_ws, lpad, coalesce, when
  
  # Cria uma view com o mesmo nome do DataFrame
  ESTABELECIMENTOS.createOrReplaceTempView("ESTABELECIMENTOS")
  EMPRESAS.createOrReplaceTempView("EMPRESAS")
  MUNICIPIOS.createOrReplaceTempView("MUNICIPIOS")
  query = f"""
            SELECT
              CONCAT(LPAD(ESTABELECIMENTOS.CNPJ_BASE, 8, '0'), LPAD(ESTABELECIMENTOS.CNPJ_ORDEM, 4, '0'), LPAD(ESTABELECIMENTOS.CNPJ_DV, 2, '0')) AS CNPJ,
              EMPRESAS.RAZAO_SOCIAL,
              ESTABELECIMENTOS.NOME_FANTASIA,
              ESTABELECIMENTOS.SITUACAO_CADASTRAL,
              ESTABELECIMENTOS.DATA_SITUACAO_CADASTRAL,
              ESTABELECIMENTOS.DATA_INICIO_ATIVIDADE,
              ESTABELECIMENTOS.CNAE_PRINCIPAL,
              ESTABELECIMENTOS.CNAE_SECUNDARIO,
              CONCAT(
                COALESCE(ESTABELECIMENTOS.TIPO_LOGRADOURO, ''),
                ' ',
                COALESCE(ESTABELECIMENTOS.LOGRADOURO, ''),
                ' ',
                COALESCE(ESTABELECIMENTOS.NUMERO, ''),
                ' ',
                COALESCE(ESTABELECIMENTOS.COMPLEMENTO, '')
              ) AS ENDERECO,
              ESTABELECIMENTOS.BAIRRO,
              MUNICIPIOS.NOME_MUNICIPIO AS CIDADE,
              ESTABELECIMENTOS.UF,
              ESTABELECIMENTOS.CEP,
              CONCAT(
                COALESCE(ESTABELECIMENTOS.DDD_CONTATO, ''),
                ' ',
                COALESCE(ESTABELECIMENTOS.TELEFONE_CONTATO, '')
              ) AS TELEFONE,
              ESTABELECIMENTOS.EMAIL
            FROM ESTABELECIMENTOS
            RIGHT JOIN EMPRESAS ON ESTABELECIMENTOS.CNPJ_BASE = EMPRESAS.CNPJ_BASE
            RIGHT JOIN MUNICIPIOS ON ESTABELECIMENTOS.CODIGO_MUNICIPIO = MUNICIPIOS.CODIGO_MUNICIPIO
            WHERE ESTABELECIMENTOS.CNAE_PRINCIPAL IN {tupla_strings}
              AND ESTABELECIMENTOS.SITUACAO_CADASTRAL IN (2, 3, 4)
            ORDER BY ESTABELECIMENTOS.CNPJ_BASE;
          """
  # Cria um DataFrame com as colunas necessárias.
  dataframe = (
      spark.sql(query)
  )
  # Exibe os primeiros 5 registros.
  # dataframe.show(5)

  # Retorna o DataFrame.
  return dataframe


In [ ]:
def salvar_df_cnae(CNAES:dict[int,str] = CNAES, opcao:int = None):
    from backup_limpeza import backup_limpeza_simples
    """
    Args:
        CNAES (dict[int,str], optional): informa um dicionário com os códigos e descrição cnae.
    Return:
        dados_pandas : salva o dataframe gerador pela função em um arquivo único csv e arquivos parquets
    """
    arquivo_csv = os.path.join(dir_dados, r"csv\BASE_RFB.csv")
    if os.path.exists(arquivo_csv):
        nome_backup = dir_dados + r"/backup/"
        if not os.path.exists(nome_backup):
            os.makedirs(nome_backup)
        backup_limpeza_simples(pasta=arquivo_csv.replace(r"BASE_RFB.csv", ""), nome_zipado=nome_backup + f"BASE_RFB_{strftime('%d-%m-%Y %H_%M_%S', localtime())}.zip")
    if opcao == 1:
        dados = None
        for cod_cnae, descricao_cnae in CNAES.items():
            if dados is None:
                
                dados = filtragem_cnae_sql(cod_cnae)
                dados = dados.withColumn(
                                "CNAE_DESCRICAO", lit(descricao_cnae.upper())
                            )
                print(f"{cod_cnae}:{descricao_cnae}")
            else:
                
                dados_incrementados = filtragem_cnae_df(cod_cnae)
                dados_incrementados = dados_incrementados.withColumn(
                    "CNAE_DESCRICAO", lit(descricao_cnae.upper())
                )
                dados = dados.union(dados_incrementados)
                dados = dados.sort("CNPJ")
                print(f"{cod_cnae}:{descricao_cnae}")
        print("passando para pandas o df spark")
        dados_pandas = dados.toPandas()
        print(len(dados_pandas.index))
        dados_pandas.to_csv(arquivo_csv, sep=";",mode="a", encoding="utf-8", index=False)
    elif opcao == 2:
        lista_cnaes = list(CNAES.keys())
        dados = filtra_cnae_lista(lista_cnaes)
        cnaes = pd.DataFrame(CNAES.items(),columns=["CNAE_PRINCIPAL", "CNAE_DESCRICAO"], dtype="string")
        dados_pandas = dados.toPandas()
        dados_pandas = pd.merge(dados_pandas, cnaes, on="CNAE_PRINCIPAL")
        dados_pandas = dados_pandas[["CNPJ","RAZAO_SOCIAL","NOME_FANTASIA",
            "SITUACAO_CADASTRAL","DATA_SITUACAO_CADASTRAL",
            "DATA_INICIO_ATIVIDADE","CNAE_PRINCIPAL","ENDERECO",
            "BAIRRO","CIDADE","UF","CEP","TELEFONE","CNAE_DESCRICAO", "EMAIL"]]
        print(len(dados_pandas.index))
        dados_pandas.to_csv(arquivo_csv, sep=";",mode="a", encoding="utf-8", index=False)
    elif opcao == 3:
        lista_cnaes = list(CNAES.keys())
        dados = filtra_cnae_lista_sql(lista_cnaes)
        cnaes = pd.DataFrame(CNAES.items(),columns=["CNAE_PRINCIPAL", "CNAE_DESCRICAO"], dtype="string")
        dados_pandas = dados.toPandas()
        dados_pandas = pd.merge(dados_pandas, cnaes, on="CNAE_PRINCIPAL")
        dados_pandas = dados_pandas[["CNPJ","RAZAO_SOCIAL","NOME_FANTASIA",
            "SITUACAO_CADASTRAL","DATA_SITUACAO_CADASTRAL",
            "DATA_INICIO_ATIVIDADE","CNAE_PRINCIPAL","ENDERECO",
            "BAIRRO","CIDADE","UF","CEP","TELEFONE","CNAE_DESCRICAO", "EMAIL"]]
        print(len(dados_pandas.index))
        dados_pandas.to_csv(arquivo_csv, sep=";",mode="a", encoding="utf-8", index=False)
    else:
        pass

In [ ]:
salvar_df_cnae(CNAES, opcao=1)

In [ ]:
lista_cnaes = list(CNAES.keys())
lista_strings = [str(cnae) for cnae in lista_cnaes]
print(lista_strings)